In [54]:
import streamlit as st
import pandas as pd
import numpy as np
import pydeck as pdk
import random
import re

In [151]:
df = pd.read_json(r"D:/2024/2110403_DSDE_Final_Project/src/data/db-data/predicted_data.json")

In [152]:
new_df = df.groupby('affiliation')['predict_gender'].value_counts().unstack(fill_value=0)
map_subject = df.groupby('affiliation')['subject_code'].value_counts().unstack(fill_value=0)

In [ ]:
new_df = df1.groupby(['affiliation', 'predict_gender']).agg(count=('predict_gender', 'count')).reset_index()
new_df

In [170]:
dfdf = df.groupby(['affiliation', 'subject_code'])['predict_gender'].value_counts().unstack(fill_value=0)

In [171]:
dfdf

predict_gender                                                   f  m
affiliation                                        subject_code      
1Department of Geography and Environmental Stud... 2105          1  0
                                                   2300          1  0
                                                   3304          1  0
1Department of Wildland Resources, Ecology Cent... 2105          1  0
                                                   2300          1  0
...                                                             .. ..
iCub Facility, Istituto Italiano di Tecnologia,... 1702          0  3
                                                   1707          0  3
                                                   1708          0  3
                                                   1709          0  3
                                                   2208          0  3

[10989 rows x 2 columns]

In [154]:
new_df['weight'] = new_df['m'] + new_df['f']

In [155]:
new_df

predict_gender,f,m,weight
affiliation,,,
"1Department of Geography and Environmental Studies, University of Colorado Colorado Springs, Colorado Springs, Colorado, USA",3,0,3
"1Department of Wildland Resources, Ecology Center, Utah State University, Logan, Utah, United States Email: jarod.raithel@aggiemail.usu.edu",3,0,3
"1Fenner School of Environment and Society, Australian National University, Canberra, Australia Email: David.freudenberger@anu.edu.au",0,6,6
"1Institut des sciences de la mer de Rimouski, Université du Québec à Rimouski, Rimouski, Québec, G5L 3A1, Canada",3,0,3
"1School of Environment and Natural Resources, The Ohio State University, Columbus, OH, United States Email: berardo.3@osu.edu",3,0,3
...,...,...,...
Yugoslavia,0,1,1
Zambia,6,15,21
"Zernike\rInstitute for Advanced Materials, University of Groningen, Nijenborgh\r4, Groningen 9747AG, The Netherlands",0,2,2


In [156]:
num_rows = new_df.shape[0]

longitudes = [random.uniform(-180, 180) for _ in range(num_rows)]
latitudes = [random.uniform(-90, 90) for _ in range(num_rows)]

new_df['longitude'] = longitudes
new_df['latitude'] = latitudes

In [ ]:
# min_count = new_df['weight'].min()
# max_count = new_df['weight'].max()

# new_df['count_normalized'] = (new_df['weight'] - min_count) / (max_count - min_count)
# print(new_df)

predict_gender                                       f   m  weight  \
affiliation                                                          
1Department of Geography and Environmental Stud...   3   0       3   
1Department of Wildland Resources, Ecology Cent...   3   0       3   
1Fenner School of Environment and Society, Aust...   0   6       6   
1Institut des sciences de la mer de Rimouski, U...   3   0       3   
1School of Environment and Natural Resources, T...   3   0       3   
...                                                 ..  ..     ...   
Yugoslavia                                           0   1       1   
Zambia                                               6  15      21   
Zernike\rInstitute for Advanced Materials, Univ...   0   2       2   
Zimbabwe                                            10  15      25   
iCub Facility, Istituto Italiano di Tecnologia,...   0  15      15   

predict_gender                                       longitude   latitude  \
affiliation 

In [157]:
new_df.columns

Index(['f', 'm', 'weight', 'longitude', 'latitude'], dtype='object', name='predict_gender')

In [158]:
new_df['weight']

affiliation
1Department of Geography and Environmental Studies, University of Colorado Colorado Springs, Colorado Springs, Colorado, USA                    3
1Department of Wildland Resources, Ecology Center, Utah State University, Logan, Utah, United States Email: jarod.raithel@aggiemail.usu.edu     3
1Fenner School of Environment and Society, Australian National University, Canberra, Australia Email: David.freudenberger@anu.edu.au            6
1Institut des sciences de la mer de Rimouski, Université du Québec à Rimouski, Rimouski, Québec, G5L 3A1, Canada                                3
1School of Environment and Natural Resources, The Ohio State University, Columbus, OH, United States Email: berardo.3@osu.edu                   3
                                                                                                                                               ..
Yugoslavia                                                                                                      

In [159]:
# new_df['radius'] = new_df['count_normalized'] * 100000000000
new_df['radius'] = new_df['weight'] * 100
# new_df['radius'] = np.clip(new_df['radius'], 0, 0000)

In [160]:
max(new_df['radius'])

17242900

In [169]:
x = new_df.reset_index()
x['affiliation']

0      1Department of Geography and Environmental Stu...
1      1Department of Wildland Resources, Ecology Cen...
2      1Fenner School of Environment and Society, Aus...
3      1Institut des sciences de la mer de Rimouski, ...
4      1School of Environment and Natural Resources, ...
                             ...                        
520                                           Yugoslavia
521                                               Zambia
522    Zernike\rInstitute for Advanced Materials, Uni...
523                                             Zimbabwe
524    iCub Facility, Istituto Italiano di Tecnologia...
Name: affiliation, Length: 525, dtype: object

In [161]:
# Define colors for each gender
base_color_f = [255, 0, 0]  # Red for 'f'
base_color_m = [0, 0, 255]  # Blue for 'm'
def calculate_color(f_count, m_count):
    total = f_count + m_count
    if total == 0:
        return [128, 128, 128]  # Neutral gray for no data
    f_ratio = f_count / total
    m_ratio = m_count / total
    blended_color = [
        int(f_ratio * base_color_f[0] + m_ratio * base_color_m[0]),
        int(f_ratio * base_color_f[1] + m_ratio * base_color_m[1]),
        int(f_ratio * base_color_f[2] + m_ratio * base_color_m[2]),
    ]
    return blended_color

new_df['color'] = new_df.apply(
    lambda row: calculate_color(row.get('f', 0), row.get('m', 0)), axis=1
)

In [ ]:
scatter_layer = pdk.Layer(
    "ScatterplotLayer",
    new_df,
    get_position="[longitude, latitude]",
    # get_color="color",
    get_color='color', #Red for clusters
    get_radius="radius",
    radius_max_pixels=50,
    opacity=0.8,
    pickable=True
)
view_state = pdk.ViewState(
    latitude=15.87,
    longitude=100.9925,
    zoom=10
)
deck = pdk.Deck(layers=[scatter_layer], initial_view_state=view_state, tooltip={"text": "{cluster}\n{subdistrict} {district}\n{timestamp}"})
deck
# st.pydeck_chart(deck)

{
  "initialViewState": {
    "latitude": 15.87,
    "longitude": 100.9925,
    "zoom": 10
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 3,
          "latitude": -33.526151973802016,
          "longitude": -168.50677033730994,
          "m": 0,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 3,
          "latitude": -2.8470969729222446,
          "longitude": -168.15667645714967,
          "m": 0,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            0,
            0,
            255
          ],
          "f": 0,
          "latitude": -23.0784091049363,
          "longitude": 116.77762570682631,
          "m": 6,
          "radius": 600,
          "weight": 6
        },
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 3,
          "latitude": 21.188835895837784,
          "longitude": -32.30807296101429,
          "m": 0,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 3,
          "latitude": 82.60230415300825,
          "longitude": 99.3290438281287,
          "m": 0,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 1,
          "latitude": 36.842401579101306,
          "longitude": 111.73153446235733,
          "m": 0,
          "radius": 100,
          "weight": 1
        },
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 3,
          "latitude": 38.92078545386991,
          "longitude": -93.89414943419271,
          "m": 0,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            0,
            0,
            255
          ],
          "f": 0,
          "latitude": 49.37357144607867,
          "longitude": 29.829523366686914,
          "m": 3,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            0,
            0,
            255
          ],
          "f": 0,
          "latitude": 8.288008511428487,
          "longitude": -74.09538249250399,
          "m": 3,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            255,
            0,
            0
          ],
          "f": 3,
          "latitude": 8.567524335540199,
          "longitude": 15.044356362948747,
          "m": 0,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            127,
            0,
            127
          ],
          "f": 3,
          "latitude": 10.46148964992527,
          "longitude": 176.38409136425918,
          "m": 3,
          "radius": 600,
          "weight": 6
        },
        {
          "color": [
            0,
            0,
            255
          ],
          "f": 0,
          "latitude": -24.59535688843343,
          "longitude": -157.67686618019715,
          "m": 3,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            0,
            0,
            255
          ],
          "f": 0,
          "latitude": -11.052245990730043,
          "longitude": -65.70035546866742,
          "m": 3,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            0,
            0,
            255
          ],
          "f": 0,
          "latitude": 11.339815360662797,
          "longitude": 142.68799337925094,
          "m": 3,
          "radius": 300,
          "weight": 3
        },
        {
          "color": [
            255,
            0,
    